In [1]:
import tensorflow as tf
tf.__version__

'2.11.0'

In [1]:
pwd


'd:\\image_classification\\notebook'

In [2]:
import os

In [4]:
os.chdir("../")

In [5]:
pwd

'd:\\image_classification'

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np

from glob import glob

In [3]:
IMAGE_SIZE = [224,224]
train_path = 'data/train'
valid_path = 'data/validation'

In [4]:
vgg16 = VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [5]:
for layer in vgg16.layers:
    print(layer)

In [6]:
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
for layer in vgg16.layers:
    print(layer.name,layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


In [8]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21,137,986
Trainable params: 6,423,298
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )

test_datagen = ImageDataGenerator(
                                    rescale = 1./255
                                )

In [13]:
pwd

'd:\\image_classification\\notebooks'

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('d:/image_classification/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 337 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('d:/image_classification/data/validation',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 59 images belonging to 2 classes.


In [16]:
import scipy

In [17]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/10
11/11 [==============================] - 63s 6s/step - loss: 3.9863 - accuracy: 0.5223 - val_loss: 2.1789 - val_accuracy: 0.4915
Epoch 2/10
11/11 [==============================] - 67s 6s/step - loss: 1.0690 - accuracy: 0.5668 - val_loss: 0.4075 - val_accuracy: 0.8305
Epoch 3/10
11/11 [==============================] - 67s 6s/step - loss: 0.4529 - accuracy: 0.7834 - val_loss: 0.4462 - val_accuracy: 0.7627
Epoch 4/10
11/11 [==============================] - 66s 6s/step - loss: 0.2518 - accuracy: 0.8932 - val_loss: 0.4023 - val_accuracy: 0.7627
Epoch 5/10
11/11 [==============================] - 66s 6s/step - loss: 0.2123 - accuracy: 0.9110 - val_loss: 0.3852 - val_accuracy: 0.8814
Epoch 6/10
11/11 [==============================] - 67s 6s/step - loss: 0.1694 - accuracy: 0.9347 - val_loss: 0.3758 - val_accuracy: 0.8644
Epoch 7/10
11/11 [==============================] - 67s 6s/step - loss: 0.1592 - accuracy: 0.9466 - val_loss: 0.4015 - val_accuracy: 0.8305
Epoch 8/10
11/11 [==

In [19]:
from tensorflow.keras.models import load_model
model.save('d:/image_classification/model/model.h5')